In [ ]:
#|default_exp consultas

In [ ]:
#|export
import re
import os
from typing import Union, Iterable
from time import time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

import requests
import pandas as pd

from fastcore.basics import store_attr, listify
from fastcore.foundation import L
from fastcore.parallel import parallel
from fastcore.xtras import Path, partialler

from validate_docbr import CPF, CNPJ

from dotenv import load_dotenv

load_dotenv()

BASEURL = "https://apinet{}/informacaoeconhecimento/entidade/receitaws/rest/1.0.0/"
AMBIENTE = {'hm', 'pd'}
TIPOS = {'cpf': "obterPessoaFisica", 'cnpj': "obterPessoaJuridica"}
CPF_SIMPLES = ('cpf', 'nome', 'nomeMae', 'dataNascimento', 'estrangeiro',
               'tituloEleitor', 'dataAtualizacao', 'dataRegistroAnatel', 
               'resultado', 'unidadeAdministrativaCodigo', 'anoObito', 'erro')
SEXO =  {1: 'Masculino', 2: 'Feminino', 9 : 'Não encontrado'}
TAMANHO = {11: 'cpf', 14: 'cnpj'}
TOKEN_URL_HM = 'https://apinethm/token'
TOKEN_URL_PD = 'https://api.anatel.gov.br/token' 
PAYLOAD = {'grant_type':'client_credentials'}
CURL = "curl -k -X POST '{}' -d 'grant_type=client_credentials' -H 'Authorization: Basic {}'"
REQUEST = "curl -k -X 'GET' '{}' -H 'accept: application/json' -H 'Authorization: Bearer {}'"


## Classe Principal para Requisição

In [ ]:
#|export
class Requisicao:
    """Classe para fazer requisições ao Web Service Receita - WS da Anatel, este encapsula o Infoconv da Receita Federal"""
    def __init__(self, 
                 cpf_usuario: Union[str, int], # CPF do usuário requisitante
                 tipo: str, # Tipo de Requisição CPF | CNPJ
                 ambiente: str = 'hm', # Ambiente onde realizar a requisição: ds: desenvolvimento, hm: homologação, su: sustentação, pd: produção
                 origem: str = None, # Texto com identificação da requisição: e.g. "Teste"
                 cache: int = 36, # Tempo de expiração do cache em meses (Produção)
    ):
        store_attr()
        self._validar_parametros()
        self.origem = self.origem[:30]
        self.cpf_usuario = ''.join(re.findall("\d", str(self.cpf_usuario)))        
        self.credentials = {}

    def _validar_parametros(self)->None:
        """Método interno de verificação e validação de parâmetros"""
        if not isinstance(self.tipo, str) or (tipo := self.tipo.lower()) not in TIPOS.keys():
            raise ValueError("Forneça uma string válida para o tipo de requisição: {CPF | CNPJ} (Case insensitive)")
        self.tipo = tipo
        self.classe = CPF() if self.tipo == 'cpf' else CNPJ()
        assert isinstance(self.origem, str), "origem não pode ficar vazio e deve ser uma string de até 30 caracteres"
        assert (ambiente := self.ambiente.lower()) in AMBIENTE, f"Ambiente inválido, escolha uma das opções {AMBIENTE}"
        if ambiente == 'pd':
            self.ambiente = ''
            self.token_url = TOKEN_URL_PD
            self.key = 'KEY_PD'
        else: 
            self.ambiente = 'hm'
            self.token_url = TOKEN_URL_HM
            self.key = 'KEY_HM'
            
    def _pad_zeros(self, identificador: Union[int, str], # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    )->str:
        """Preenche o identificador com zeros à esquerda até o comprimento total: {CPF:11, CNPJ:15}"""
        identificador = ''.join(re.findall("\d", str(identificador)))
        if self.tipo == 'cpf':
            return identificador.zfill(11)
        elif self.tipo == 'cnpj':
            return identificador.zfill(14)
        return ''
    

    def gerar_token(self)->None:
        """Gera o token de requisição com validade de 1h à partir da chave de acesso do usuário
        As chaves de acesso são acessadas como variáveis de ambiente: KEY_HM | KEY_PD
        """
        if start := self.credentials.get('time'):
            if time() - start < 3600 and 'token' in self.credentials:
                return    
        headers = {"Authorization": f"Basic {os.getenv(self.key)}"}
        r = requests.post(self.token_url, json=PAYLOAD, verify=False, headers=headers)
        if (status := r.status_code) != 200:
            raise ValueError(f"A requisição de token não foi bem-sucedida: {status}")
        self.credentials['token'] = r.json().get('access_token')
        self.credentials['time'] = time()        
        

    def _formatar_url(self,
                  identificador, # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    )-> str: # Url formatada para a requisição GET REST
        """Recebe o identificador CPF|CNPJ e formata retorna a url formatada da requisição"""
        req_TIPO = TIPOS[self.tipo]
        suffix = ''
        if self.cache is not None:
            try:
                cache = int(self.cache)
                assert cache >= 0, "Tempo de expiração do cache inválido, escolha um número inteiro maior que zero"
            except ValueError as e:
                raise ValueError("Valor inválido de expira_cache, escolha um número inteiro maior que zero") from e
            req_TIPO += 'IgnoraCacheAntigo'
            suffix = f'&mesesExpiraCache={cache}'

        return f'{BASEURL.format(self.ambiente)}{req_TIPO}?{self.tipo}={identificador}&cpfUsuario={self.cpf_usuario}&origem={self.origem}{suffix}'

    @staticmethod
    def nivelar_json(json_dict: dict, # Dicionário no formato json retornado pela requisição
    )-> dict: #Dicionário nivelado para formato tabular
        """Recebe um dicionário json com diferentes níveis e retorna os registros em um único nível"""
        d = {}
        for k,v in json_dict.items():
            if isinstance(v, list): 
                v = '|'.join(v)
            if isinstance(v, dict):
                d.update({f'{k}.{sk}': sv for sk, sv in v.items()})
                continue
            d[k] = v
        return {k:v.lstrip().rstrip() if isinstance(v, str) else v for k, v in d.items()}

    @staticmethod
    def _get_request(url: str, # Url formatada da requisição 
                     token: str, # Token de autorização
    )-> dict:
        """Efetua a requisição na url and retorna a resposta json"""
        headers = {'accept' : 'application/json', 
                   'Authorization' : f'Bearer {token}'}
        r = requests.get(url, headers=headers, verify=False)
        if r.status_code == 200 and r.headers['content-type'] == 'application/json':
            return Requisicao.nivelar_json(r.json())
        return {}

    def consultar(self, 
                 identificador: Union[str, int], # Identificador da requisição: CPF ou CNPJ de acordo com o tipo
    )-> dict: # Dicionário com o resultado da requisição
        """Efetua a requisição do identificador e retorna um dicionário"""
        identificador = self._pad_zeros(identificador)
        if not self.classe.validate(identificador):
            raise ValueError(f"{self.tipo.upper()} Inválido! Verifique o identificador digitado: {identificador}")
        self.gerar_token()
        return Requisicao._get_request(self._formatar_url(identificador), self.credentials['token'])
    
    def consultar_em_lote(self, 
                      identificadores: Iterable[Union[str, int]], # Lista com os identificadores: CPF ou CNPJ
                      n_workers: int = 8, # Número de requisições a serem efetuadas em paralelo
    )->Iterable[dict]: # Lista com os dicionários resultantes das requisições
        """Efetua a requisição da lista de ids identificadores de forma paralelizada por threads
        Filtra os identificadores válidos primeiramente antes de fazer a requisição. 
        Os identificadores não retornados foram considerados inválidos
        """
        identificadores = L(listify(identificadores)).map(self._pad_zeros).filter(self.classe.validate)
        if not identificadores:
            return L()
        self.gerar_token()
        urls = identificadores.map(self._formatar_url)
        func = partialler(Requisicao._get_request, token=self.credentials['token'])
        return parallel(func, urls, n_workers=n_workers, threadpool=True, progress=True)        

In [ ]:
#|export
def salvar_requisicao(results: Iterable, # Lista com o retorno das requisições
                 saida: str, # Nome do Arquivo de Saída
)->None:
    """Salva a lista de requisições `results` no arquivo `saida`"""
    df = pd.DataFrame(results)
    if saida is None:
        saida = Path.cwd() / 'resultados.csv'
    try:
        saida = Path(saida)
    except TypeError as e:
        raise TypeError("Verifique o caminho do arquivo de saída digitado!") from e

    match saida.suffix:
        case '.csv' | '.txt':
            df.to_csv(saida, index=False)
        case '.xlsx':
            df.to_excel(saida, index=False, engine='openpyxl')
        case '.json':
            df.to_json(saida)
        case '.md':
            df.to_markdown(saida, index=False)
        case '.html':
            df.to_html(saida, index=False)
        case _:
            df.to_csv(saida, index=False)

def requisitar_em_lote(entrada: str, # Arquivo texto de entrada: 1 CPF | CNPJ por linha ou objeto python iterável
                       cpf_usuario: str, # CPF do usuário requisitante
                       tipo: str, # Tipo de Requisição CPF | CNPJ
                       origem: str, # Texto com identificação da requisição: e.g. 'Teste'
                       ambiente: str = 'hm', # Ambiente onde realizar a requisição: hm | pd 
                       cache: int = 36, # Tempo de expiração do cache em meses 
                       saida: str = None, # Arquivo de saída da requisição
                       n_workers: int = 2, # Número de requisições a serem efetuadas em paralelo
)->pd.DataFrame:
    """Lê o arquivo `entrada` com um CPF | CPNJ por linha ou o objeto python iterável.
    Faz a requisição no `ambiente` do receita-ws e salva os resultados em `saida`
    """
    try:
        conteudo = Path(entrada).readlines()
    except TypeError:
        conteudo = listify(entrada)
    req = Requisicao(cpf_usuario, tipo, ambiente, origem, cache)
    resultado = req.consultar_em_lote(conteudo, n_workers)
    salvar_requisicao(resultado, saida)
    return pd.DataFrame(resultado)

In [ ]:
folder = Path.cwd().parent / 'dados'
cpf_list = pd.read_excel(folder / 'Entidades_AFFO_CPF.xlsx')
cpf_list.head()

,Unnamed: 0,Ordem,Tipo_Ident,CNPJ_CPF,Nome_Entidade,Valido
0,0,718,CPF,33481695268,RUBEM MORAIS DE LIMA,True
1,1,838,CPF,47819847034,LUIS ROBERTO CUNHA REY,True
2,2,971,CPF,18876126880,SIDNEY SOUZA DA SILVA,True
3,3,974,CPF,58201343204,VILSON ANTUNES MAXIMIANO,True
4,4,1020,CPF,21996857134,LUIZ CARLOS DA SILVA SANTOS,True


In [ ]:
cnpj = pd.read_excel(folder / 'Entidades_AFFO_CNPJ.xlsx')
cnpj.head()

,Ordem,Tipo_Ident,CNPJ_CPF,Nome_Entidade,Valido
0,1,CNPJ,76535764000143,Oi S.a. - em Recuperacao Judicial,True
1,2,CNPJ,2558157000162,TELEFONICA BRASIL S.A.,True
2,3,CNPJ,2421421000111,TIM S A,True
3,4,CNPJ,40432544000147,CLARO S.A.,True
4,5,CNPJ,43663075000165,WINITY II TELECOM LTDA,True


In [ ]:
Path(folder / 'cpf.csv').write_text('\n'.join([str(c) for c in cpf_list.head(10).CNPJ_CPF]))

118

In [ ]:
Path(folder / 'cnpj.csv').write_text('\n'.join([str(c) for c in cnpj.head(10).CNPJ_CPF]))

143

## Exemplos



### CPF

In [ ]:
from fastcore.test import ExceptionExpected, test_eq

In [ ]:
cpf = CPF()

In [ ]:
cpf_usuario = cpf.generate()
identificador = cpf.generate()
tipo = 'cpf'
origem = "TestePython"
ambiente = 'hm'
cache = 6

In [ ]:
requisitar_em_lote(folder / 'cpf.csv', cpf_usuario, tipo='cpf', origem='Teste Python')

,cpf,nome,situacaoCadastral.codigo,situacaoCadastral.valor,paisResidencia.residenteExterior,paisResidencia.codigoPais,nomeMae,dataNascimento,sexo.codigo,sexo.valor,...,telefone.numero,unidadeAdministrativaCodigo,anoObito,estrangeiro,tituloEleitor,dataAtualizacao,dataRegistroAnatel,resultado,erro,ocupacao.naturezaOcupacaoDescricao
0,33481695268,Ls Ifmqinlkurrssezna,2,Suspensa,True,0,Cqyqxgqgumqinlkurrssezna,1997-09-12,1,Masculino,...,97219728,7439828,0,False,0000000000000,1937-04-06,2022-09-27,CPF encontrado,,NaN
1,47819847034,Tuxktnciwueq Koithctjx,0,Regular,True,0,Abpcgqu Koithctjx,1980-01-13,9,Sem informacao,...,15966520,6976220,0,False,0000000000000,1928-09-13,2022-09-27,CPF encontrado,,Membro ou servidor público da administração di...
2,18876126880,Nyopkqyx Rwdykddcmmro,8,Nula,True,0,Edcgltyiqsprxfddcmmro,1986-12-22,9,Sem informacao,...,32104647,5700228,0,False,0000000000000,1965-09-17,2022-09-27,CPF encontrado,,NaN
3,58201343204,J Y Ubevpjgbcvzubibdhvje,0,Regular,True,0,Razgqipmdwypegctupbrzubibdhvje,1924-12-17,9,Sem informacao,...,60991414,0021524,0,False,0000000000000,1965-05-22,2022-09-27,CPF encontrado,,NaN
4,21996857134,Zzhpkuwfqwqlckrmbguszqfcfbb,4,Pendente de Regularizacao,True,0,Byezpzzzlckrmbguszqfcfbb,1963-12-08,2,Feminino,...,03848341,8697223,0,False,0000000000000,1930-03-25,2022-09-27,CPF encontrado,,Membro ou servidor público da administração di...
5,25367495842,Tdexnrovkjpylffcuua,8,Nula,True,0,Wycxfbgojbksdxzjllffcuua,1985-07-31,1,Masculino,...,63426936,6092170,0,False,0000000000000,2006-08-24,2022-09-27,CPF encontrado,,NaN
6,58948023691,Cxdb Nqfhnlutcni Cti,9,Cancelada de Oficio,False,0,NaN,1961-09-21,9,Sem informacao,...,01723291,1333898,0,True,0000000000000,1962-11-25,2022-09-27,CPF encontrado,,Empregado de instituições financeiras públicas...
7,09465085693,Qzztsrsjcnsmtob Nzkiktjm,5,Cancelada por Multiplicidade,False,0,Pig Fdidrejqnfqcjlqttsiyxsv,1972-08-18,1,Masculino,...,96816835,6426935,0,True,0000000000000,1948-05-14,2022-09-27,CPF encontrado,,NaN
8,83792430487,Vcm Ntdwndjxdxsxovdy,1,Cancelada por Encerramento de Espolio,False,0,Efmyumjtdwndjxdxsxovdy,1932-02-23,1,Masculino,...,92991603,3370473,0,True,0000000000000,2007-02-03,2022-09-27,CPF encontrado,,NaN
9,58066098200,Mvggvgmtra Wpgxdrrgeu,2,Suspensa,True,0,Jlgabppbailxftfdevggl,1938-10-04,9,Sem informacao,...,85120080,1864416,0,False,0000000000000,1954-02-15,2022-09-27,CPF encontrado,,NaN


In [ ]:
requisitar_em_lote(folder / 'cnpj.csv', cpf_usuario, tipo='cnpj', origem='Teste Python')

,cnpj,matriz,nomeEmpresarial,nomeFantasia,situacaoCadastral.codigo,situacaoCadastral.valor,situacaoCadastral.dataAlteracao,naturezaJuridica.codigo,dataAbertura,cnaePrincipal,...,contador.cpfContador,sociedade.socio,dataRegistroAnatel,resultado,erro,endereco.codigoMunicipio,naturezaJuridica.descricao,telefone2.ddd,telefone2.numero,contador.ufCrcContadorPj
0,76535764000143,True,Prug P;u'ezbinnvqzumekfwtlrlelxqj,Dg,02,Ativa,1967-12-20T00:00:00,5827,1951-01-29,0633682,...,22133133186,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2022-09-28,CNPJ encontrado,,NaN,NaN,NaN,NaN,NaN
1,02558157000162,True,Gaq Hzeeaxghgdcxspx?y.,,03,Suspensa,1990-04-24T00:00:00,7153,1932-03-24,6075918,...,21313233285,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2022-09-28,CNPJ encontrado,,NaN,NaN,NaN,NaN,NaN
2,02421421000111,False,Xs N Kf,,03,Suspensa,1991-03-07T00:00:00,0064,1964-04-03,2418829,...,33111332349,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2022-09-28,CNPJ encontrado,,3101508.0,NaN,NaN,NaN,NaN
3,40432544000147,True,Claro S.a.,Claro,02,Ativa,2020-07-16T00:00:00,2054,1992-04-23,6120501,...,90656920106,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2021-09-02,CNPJ encontrado,,3550308.0,SOCIEDADE ANONIMA FECHADA,NaN,NaN,NaN
4,43663075000165,True,Q Lgvgdukdxdroxjusxpne#,,08,Baixada,1970-02-14T00:00:00,8233,1970-02-14,3596772,...,21133121314,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2022-09-28,CNPJ encontrado,,2210391.0,NaN,NaN,NaN,NaN
5,72820822000120,True,Igulqpcanbwluymwvrh Sfui,Eaf,02,Ativa,1987-02-06T00:00:00,8171,1955-02-24,7649519,...,23213213349,"[{'tipo': {'codigo': 1, 'valor': 'Socio PJ'}, ...",2022-09-28,CNPJ encontrado,,NaN,NaN,NaN,NaN,NaN
6,00497373000110,False,Tljcmjtzzqbkqnvlvvivyipgcsgzotux.,Nbvifgzpgzoahcb,08,Baixada,1947-09-30T00:00:00,5600,1908-08-27,7913441,...,23333222175,"[{'tipo': {'codigo': 1, 'valor': 'Socio PJ'}, ...",2022-09-28,CNPJ encontrado,,2102408.0,NaN,59,32033730,NaN
7,05958690000100,True,Sbctbppezxlgncbdrlqlnbsruuyagscnebhfpx%,,03,Suspensa,1966-12-28T00:00:00,3541,1977-09-08,1731080,...,31133223249,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2022-09-28,CNPJ encontrado,,NaN,NaN,90,25937614,NaN
8,76535764032690,False,Abpc!e E(vnnwqiplbubmhrrmhtnwbomc,,02,Ativa,1963-04-03T00:00:00,1203,1959-05-06,5419068,...,23232333354,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2022-09-28,CNPJ encontrado,,NaN,NaN,NaN,NaN,ES
9,04601397000128,True,Eibtddutncj Nvntnmzhcgkxddv Edgwtijjtfs)u$,Xwjcazyn,08,Baixada,1927-10-02T00:00:00,9861,1927-10-02,8827610,...,31332111378,"[{'tipo': {'codigo': 2, 'valor': 'Socio PF'}, ...",2022-09-28,CNPJ encontrado,,NaN,NaN,NaN,NaN,NaN


In [ ]:
req = Requisicao(cpf_usuario, tipo, ambiente, origem, cache)

In [ ]:
req._formatar_url(identificador)

'https://apinethm/informacaoeconhecimento/entidade/receitaws/rest/1.0.0/obterPessoaFisicaIgnoraCacheAntigo?cpf=82344737510&cpfUsuario=79148622400&origem=TestePython&mesesExpiraCache=6'

In [ ]:
r = req.consultar(identificador)

In [ ]:
del r['cpf'] # Não expõe o CPF no código
r

{'nome': '',
 'nomeMae': '',
 'dataNascimento': '1900-01-01',
 'unidadeAdministrativaCodigo': '',
 'anoObito': 0,
 'estrangeiro': False,
 'tituloEleitor': '',
 'dataAtualizacao': '1900-01-01',
 'dataRegistroAnatel': '1900-01-01',
 'resultado': 'Erro INFOCONV: CPF - Erro 04 - CPF não encontrado.(CPF: 82344737510)',
 'erro': 'E04'}

In [ ]:
cpfs = req.consultar_em_lote([cpf.generate() for _ in range(10)])

In [ ]:
cpfs = cpfs.map(lambda x: {k:v for k,v in x.items() if k != 'cpf'}) # Não expõe os CPFs
cpfs

(#10) [{'nome': '', 'nomeMae': '', 'dataNascimento': '1900-01-01', 'unidadeAdministrativaCodigo': '', 'anoObito': 0, 'estrangeiro': False, 'tituloEleitor': '', 'dataAtualizacao': '1900-01-01', 'dataRegistroAnatel': '1900-01-01', 'resultado': 'Erro INFOCONV: CPF - Erro 04 - CPF não encontrado.(CPF: 09885969039)', 'erro': 'E04'},{'nome': '', 'nomeMae': '', 'dataNascimento': '1900-01-01', 'unidadeAdministrativaCodigo': '', 'anoObito': 0, 'estrangeiro': False, 'tituloEleitor': '', 'dataAtualizacao': '1900-01-01', 'dataRegistroAnatel': '1900-01-01', 'resultado': 'Erro INFOCONV: CPF - Erro 04 - CPF não encontrado.(CPF: 44848530431)', 'erro': 'E04'},{'nome': '', 'nomeMae': '', 'dataNascimento': '1900-01-01', 'unidadeAdministrativaCodigo': '', 'anoObito': 0, 'estrangeiro': False, 'tituloEleitor': '', 'dataAtualizacao': '1900-01-01', 'dataRegistroAnatel': '1900-01-01', 'resultado': 'Erro INFOCONV: CPF - Erro 04 - CPF não encontrado.(CPF: 55153449793)', 'erro': 'E04'},{'nome': 'Nvzafhcphilbmb Qe

In [ ]:
pd.DataFrame(cpfs)

,nome,nomeMae,dataNascimento,unidadeAdministrativaCodigo,anoObito,estrangeiro,tituloEleitor,dataAtualizacao,dataRegistroAnatel,resultado,...,ocupacao.exercicioOcupacao,endereco.logradouro,endereco.numero,endereco.cep,endereco.bairro,endereco.codigoMunicipio,endereco.nomeMunicipio,endereco.uf,telefone.ddd,telefone.numero
0,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Nvzafhcphilbmb Qefqdrifmnjkmhqrcbsw,Xyqdepwclxfzrqvcfbw,1923-09-14,9764733,0,False,0000000000000,1950-08-20,2022-10-05,CPF encontrado,...,6642.0,Kgeoc Cnbt Wfpryzfw,051,53934634,Skemoxw,4309159.0,Kruop,MT,0013,99434634
4,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,,,1900-01-01,,0,False,,1900-01-01,1900-01-01,Erro INFOCONV: CPF - Erro 04 - CPF não encontr...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Hpotwubajhbzrv Cybndfqgfakon,Bxilfnnegtxi Evauixhkn,1950-01-10,8003358,0,False,0000000000000,1976-04-13,2022-10-05,CPF encontrado,...,4758.0,Qnn Hamozigexmuz,"0770,",83843778,NaN,2412906.0,Bhdgcttea,PR,0008,88294758


### CNPJ


CNPJ somente números

In [ ]:
tipo = 'cnpj'
req = Requisicao(cpf_usuario, tipo, ambiente, origem)
req.consultar('00280273000137')

{'cnpj': '00280273000137',
 'matriz': False,
 'nomeEmpresarial': 'Cyqrunvu Jxt Bglhignngsnsrubjdhyfa',
 'nomeFantasia': 'Ah Vgfmaquyhfjhkska',
 'situacaoCadastral.codigo': '02',
 'situacaoCadastral.valor': 'Ativa',
 'situacaoCadastral.dataAlteracao': '1996-01-02T00:00:00',
 'naturezaJuridica.codigo': '1940',
 'dataAbertura': '1930-09-16',
 'cnaePrincipal': '0449988',
 'cnaeSecundario': '9721477|0420288|0421088|0431888|0479080|0623080|1112877|2417380|2417377|2444089|2448389|2448380|2448377|2450589|2450580|2451388|3010677|4001288|4002088|4110888|4421288|5111188|5118980|5299082|5299077|5538980|5538977|5549288|6010288|6098677|7310788|7311588|7320488|7328077',
 'endereco.logradouro': 'Udnjnfa Gvldxeyks',
 'endereco.numero': '0348',
 'endereco.cep': '67806980',
 'endereco.bairro': 'Pxqbywwanotqnv Krjh Ki',
 'endereco.codigoMunicipio': 5106703,
 'endereco.nomeMunicipio': 'Vjjfws',
 'endereco.uf': 'SE',
 'telefone1.ddd': '70',
 'telefone1.numero': '48890992',
 'telefone2.ddd': '70',
 'telefone

CNPJ com separadores

In [ ]:
req.consultar('02.030.715/0001-12')

{'cnpj': '02030715000112',
 'matriz': True,
 'nomeEmpresarial': 'H Qyccquxlammrnsnssjydjvwdhlqqhjeaee',
 'nomeFantasia': 'Fwcfey',
 'situacaoCadastral.codigo': '03',
 'situacaoCadastral.valor': 'Suspensa',
 'situacaoCadastral.dataAlteracao': '1938-07-30T00:00:00',
 'naturezaJuridica.codigo': '3525',
 'dataAbertura': '1906-03-01',
 'cnaePrincipal': '9755621',
 'endereco.logradouro': 'Vccqrn H Dctuseqytj8jgkmthhqu[m)oxrvu',
 'endereco.numero': '31',
 'endereco.complemento': "Luug'mettlqhh'",
 'endereco.cep': '81312361',
 'endereco.bairro': 'Ixjxeuy',
 'endereco.codigoMunicipio': 4301073,
 'endereco.nomeMunicipio': 'Ezfakyin',
 'endereco.uf': 'ES',
 'telefone1.ddd': '82',
 'telefone1.numero': '34464555',
 'telefone2.ddd': '82',
 'telefone2.numero': '34463374',
 'email': "rohyx'rgqowo<lxx,bd",
 'responsavel.cpf': '50368609502',
 'responsavel.nome': 'Phj Bg Lzdnugmeinpqdrv',
 'capitalSocial': '1',
 'porte.codigo': 3,
 'porte.valor': 'Empresa de pequeno porte',
 'opcaoSimples.opcaoSimples': 

## Testes Unitários

In [ ]:
from itertools import product

In [ ]:
with ExceptionExpected(ex=AssertionError, regex= 'origem não pode ficar vazio e deve ser uma string de até 30 caracteres'):
    Requisicao(cpf_usuario=cpf_usuario, tipo=tipo)

**origem não é uma string**

In [ ]:
with ExceptionExpected(ex=AssertionError, regex= 'origem não pode ficar vazio e deve ser uma string de até 30 caracteres'):
    Requisicao(cpf_usuario=cpf_usuario, tipo=tipo, origem = 50)

**ambiente inválido**

In [ ]:
with ExceptionExpected(ex=AssertionError, regex="Ambiente inválido"):
    Requisicao(cpf_usuario=cpf_usuario, tipo = tipo, origem='Teste', ambiente='ts')

**Teste de formatação da url correta de acordo com o ambiente**

In [ ]:
origem = 'Teste'
for ambiente in AMBIENTE:
    for tipo in ('cpf', 'cnpj'):
        req_type = 'obterPessoaFisicaIgnoraCacheAntigo' if tipo == 'cpf' else 'obterPessoaJuridicaIgnoraCacheAntigo'
        req = Requisicao(cpf_usuario=cpf_usuario, tipo=tipo, origem=origem, ambiente=ambiente)
        url = req._formatar_url(identificador)
        amb = '' if ambiente == 'pd' else ambiente
        test_eq(url, f'{BASEURL.format(amb)}{req_type}?{tipo}={identificador}&cpfUsuario={cpf_usuario}&origem={origem}&mesesExpiraCache=36')

**Teste de ambientes x tipos de requisição**

In [ ]:
origem = 'Teste'
cache = 3
for (ambiente, tipo) in product(AMBIENTE, TIPOS):
    req = Requisicao(cpf_usuario=cpf_usuario,
                     tipo=tipo,
                     ambiente=ambiente, 
                     origem=origem, 
                     cache=cache)
    id_ = identificador if tipo == 'cpf' else '02030715000112'
    url = req._formatar_url(id_)
    if ambiente == 'pd': 
        ambiente = ''
    test_eq(url, f'{BASEURL.format(ambiente)}{TIPOS[tipo]}IgnoraCacheAntigo?{tipo}={id_}&cpfUsuario={cpf_usuario}&origem={origem}&mesesExpiraCache={cache}')

**Teste de ambientes x tipos de requisição - sem cache**

In [ ]:
origem = 'Teste'
cache = None
for (ambiente, tipo) in product(AMBIENTE, TIPOS):
    req = Requisicao(cpf_usuario=cpf_usuario,
                     tipo=tipo,
                     ambiente=ambiente, 
                     origem=origem, 
                     cache=cache)
    id_ = identificador if tipo == 'cpf' else '02030715000112'
    url = req._formatar_url(id_)
    if ambiente == 'pd': 
        ambiente = ''
    test_eq(url, f'{BASEURL.format(ambiente)}{TIPOS[tipo]}?{tipo}={id_}&cpfUsuario={cpf_usuario}&origem={origem}')

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()